In [25]:
"""
STEP File Volume Calculator for Metal Parts
Calculates volume and weight from STEP files
"""

from OCP.STEPControl import STEPControl_Reader
from OCP.IFSelect import IFSelect_ReturnStatus
from OCP.GProp import GProp_GProps
from OCP.BRepGProp import BRepGProp
from OCP.TopoDS import TopoDS_Shape
import os


def calculate_volume_from_step(step_file_path):
    """
    Calculate the volume of a 3D model from a STEP file.
    
    Args:
        step_file_path (str): Path to the STEP file
        
    Returns:
        float: Volume in cubic millimeters (mm³)
        
    Raises:
        FileNotFoundError: If the STEP file doesn't exist
        ValueError: If the STEP file cannot be read or processed
    """
    # Check if file exists
    if not os.path.exists(step_file_path):
        raise FileNotFoundError(f"STEP file not found: {step_file_path}")
    
    # Initialize STEP reader
    reader = STEPControl_Reader()
    
    # Read the STEP file
    status = reader.ReadFile(step_file_path)
    
    if status != IFSelect_ReturnStatus.IFSelect_RetDone:
        raise ValueError(f"Error reading STEP file: {step_file_path}")
    
    # Transfer the contents to the shape
    reader.TransferRoots()
    shape = reader.OneShape()
    
    if shape.IsNull():
        raise ValueError("No valid shape found in STEP file")
    
    # Calculate volume using GProp
    props = GProp_GProps()
    BRepGProp.VolumeProperties_s(shape, props)
    
    volume = props.Mass()  # Mass() returns the volume for VolumeProperties
    
    return volume


def calculate_volume_xyz(x, y, z):
    """
    Calculate volume from dimensions (length × width × height).
    Simple geometric calculation for rectangular parts.
    
    Args:
        x (float): Length in millimeters (mm)
        y (float): Width in millimeters (mm)
        z (float): Height in millimeters (mm)
        
    Returns:
        float: Volume in cubic millimeters (mm³)
    """
    volume_mm3 = x * y * z
    return volume_mm3


def calculate_bounding_box_volume_from_step(step_file_path):
    """
    Calculate the bounding box volume (x × y × z) from a STEP file.
    This calculates the volume of the smallest box that contains the entire part.
    
    Args:
        step_file_path (str): Path to the STEP file
        
    Returns:
        dict: Dictionary containing bounding box dimensions and volume
        
    Raises:
        FileNotFoundError: If the STEP file doesn't exist
        ValueError: If the STEP file cannot be read or processed
    """
    from OCP.Bnd import Bnd_Box
    from OCP.BRepBndLib import BRepBndLib
    
    # Check if file exists
    if not os.path.exists(step_file_path):
        raise FileNotFoundError(f"STEP file not found: {step_file_path}")
    
    # Initialize STEP reader
    reader = STEPControl_Reader()
    
    # Read the STEP file
    status = reader.ReadFile(step_file_path)
    
    if status != IFSelect_ReturnStatus.IFSelect_RetDone:
        raise ValueError(f"Error reading STEP file: {step_file_path}")
    
    # Transfer the contents to the shape
    reader.TransferRoots()
    shape = reader.OneShape()
    
    if shape.IsNull():
        raise ValueError("No valid shape found in STEP file")
    
    # Calculate bounding box
    bbox = Bnd_Box()
    BRepBndLib.Add_s(shape, bbox)
    
    # Get bounding box dimensions
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
    
    # Calculate dimensions
    x_dimension = xmax - xmin
    y_dimension = ymax - ymin
    z_dimension = zmax - zmin
    
    # Calculate bounding box volume
    bbox_volume = x_dimension * y_dimension * z_dimension
    
    return {
        'x_min': xmin,
        'y_min': ymin,
        'z_min': zmin,
        'x_max': xmax,
        'y_max': ymax,
        'z_max': zmax,
        'x_dimension': x_dimension,
        'y_dimension': y_dimension,
        'z_dimension': z_dimension,
        'bounding_box_volume': bbox_volume
    }


def calculate_weight(volume_mm3, density_g_cm3):
    """
    Calculate weight from volume and material density.
    
    Args:
        volume_mm3 (float): Volume in cubic millimeters (mm³)
        density_g_cm3 (float): Material density in g/cm³
        
    Returns:
        dict: Dictionary containing weight in different units
    """
    # Convert mm³ to cm³ (1 cm³ = 1000 mm³)
    volume_cm3 = volume_mm3 / 1000.0
    
    # Calculate weight in grams
    weight_g = volume_cm3 * density_g_cm3
    
    # Calculate weight in kilograms
    weight_kg = weight_g / 1000.0
    
    return {
        'grams': weight_g,
        'kilograms': weight_kg,
        'pounds': weight_kg * 2.20462  # Convert to pounds
    }


# Common material densities (g/cm³)
MATERIAL_DENSITIES = {
    'aluminum': 2.70,
    'steel': 7.85,
    'stainless_steel': 8.00,
    'brass': 8.50,
    'copper': 8.96,
    'titanium': 4.51,
    'cast_iron': 7.20,
    'bronze': 8.80,
    'zinc': 7.14,
    'magnesium': 1.74,
}


def process_step_file(step_file_path, material='steel'):
    """
    Complete workflow: Read STEP file and calculate both actual and bounding box volumes.
    
    Args:
        step_file_path (str): Path to the STEP file
        material (str): Material type (default: 'steel')
        
    Returns:
        dict: Dictionary with both volume types and weight information
    """
    try:
        # Calculate actual volume
        actual_volume_mm3 = calculate_volume_from_step(step_file_path)
        
        # Calculate bounding box volume (x × y × z)
        bbox_info = calculate_bounding_box_volume_from_step(step_file_path)
        bbox_volume_mm3 = bbox_info['bounding_box_volume']
        
        # Calculate internal box volume (bounding_box / 5000)
        internal_box_v = bbox_volume_mm3 / 5000000
        
        # Get material density
        if isinstance(material, str):
            density = MATERIAL_DENSITIES.get(material.lower(), MATERIAL_DENSITIES['steel'])
        else:
            density = material
        
        # Calculate weight based on actual volume
        weight_actual = calculate_weight(actual_volume_mm3, density)
        
        # Calculate weight based on bounding box volume
        weight_bbox = calculate_weight(bbox_volume_mm3, density)
        
        max_value = max(internal_box_v, weight_actual['kilograms'])
        return {
            'success': True,
            'file': step_file_path,
            'x_dimension': bbox_info['x_dimension'],
            'y_dimension': bbox_info['y_dimension'],
            'z_dimension': bbox_info['z_dimension'],
            'actual_volume_mm3': actual_volume_mm3,
            'actual_volume_cm3': actual_volume_mm3 / 1000.0,
            'bounding_box_volume_mm3': bbox_volume_mm3,
            'bounding_box_volume_cm3': bbox_volume_mm3 / 1000.0,
            'internal_box_v': internal_box_v,
            'material': material if isinstance(material, str) else f'custom ({density} g/cm³)',
            'density_g_cm3': density,
            'actual_weight_g': weight_actual['grams'],
            'actual_weight_kg': weight_actual['kilograms'],
            'actual_weight_lbs': weight_actual['pounds'],
            'bbox_weight_g': weight_bbox['grams'],
            'bbox_weight_kg': weight_bbox['kilograms'],
            'bbox_weight_lbs': weight_bbox['pounds'],
            'max_internal_or_weight': max_value

        }
    
    except Exception as e:
        return {
            'success': False,
            'file': step_file_path,
            'error': str(e)
        }



In [27]:
    # Example 1: Calculate actual volume from STEP file
step_file = "101833-001-A.step"  # Replace with your file path# Example 1: Calculate both actual and bounding box volumes from STEP file
    
result = process_step_file(step_file, material='aluminum')
    
if result['success']:
        print(f"File: {result['file']}")
        print(f"Material: {result['material']}")
        print(f"Density: {result['density_g_cm3']} g/cm³")
        print("\n" + "-"*50)
        
        print("\nBounding Box Dimensions:")
        print(f"  X × Y × Z: {result['x_dimension']/10:.2f} × {result['y_dimension']/10:.2f} × {result['z_dimension']/10:.2f} cm")
        
        print("\nBounding Box Volume (x × y × z):")
        print(f"  Volume: ({result['bounding_box_volume_cm3']:.2f} cm³)")
        print("\nInternal BV/5000 (bv/5000):")
        print(f"  BV/5000: ({result['bounding_box_volume_cm3']/5000:.4f})")
        
        print("\nActual Part Volume:")
        print(f"  Volume: {result['actual_volume_cm3']:.2f} cm³")
        print(f"  Weight: {result['actual_weight_kg']:.4f} kg")


        print(f"\nMax Value:")
        print(f"  max(internal_box_v, actual_weight_kg) = {result['max_internal_or_weight']:.4f}")
        print(f"  (Comparing {result['internal_box_v']:.4f} vs {result['actual_weight_kg']:.4f})")

        
        
else:
        print(f"Error: {result['error']}")
    
print("\n" + "="*50 + "\n")

File: 101833-001-A.step
Material: aluminum
Density: 2.7 g/cm³

--------------------------------------------------

Bounding Box Dimensions:
  X × Y × Z: 11.86 × 11.16 × 0.32 cm

Bounding Box Volume (x × y × z):
  Volume: (42.38 cm³)

Internal BV/5000 (bv/5000):
  BV/5000: (0.0085)

Actual Part Volume:
  Volume: 33.63 cm³
  Weight: 0.0908 kg

Max Value:
  max(internal_box_v, actual_weight_kg) = 0.0908
  (Comparing 0.0085 vs 0.0908)




In [28]:
step_file = "102210-001-A.step"  # Replace with your file path# Example 1: Calculate both actual and bounding box volumes from STEP file


result = process_step_file(step_file, material='aluminum')
    
if result['success']:
        print(f"File: {result['file']}")
        print(f"Material: {result['material']}")
        print(f"Density: {result['density_g_cm3']} g/cm³")
        print("\n" + "-"*50)
        
        print("\nBounding Box Dimensions:")
        print(f"  X × Y × Z: {result['x_dimension']/10:.2f} × {result['y_dimension']/10:.2f} × {result['z_dimension']/10:.2f} cm")
        
        print("\nBounding Box Volume (x × y × z):")
        print(f"  Volume: ({result['bounding_box_volume_cm3']:.2f} cm³)")
        print("\nInternal BV/5000 (bv/5000):")
        print(f"  BV/5000: ({result['bounding_box_volume_cm3']/5000:.4f})")
        
        print("\nActual Part Volume:")
        print(f"  Volume: {result['actual_volume_cm3']:.2f} cm³")
        print(f"  Weight: {result['actual_weight_kg']:.4f} kg")


        print(f"\nMax Value:")
        print(f"  max(internal_box_v, actual_weight_kg) = {result['max_internal_or_weight']:.4f}")
        print(f"  (Comparing {result['internal_box_v']:.4f} vs {result['actual_weight_kg']:.4f})")

        
        
else:
        print(f"Error: {result['error']}")
    
print("\n" + "="*50 + "\n")

File: 102210-001-A.step
Material: aluminum
Density: 2.7 g/cm³

--------------------------------------------------

Bounding Box Dimensions:
  X × Y × Z: 17.70 × 2.70 × 7.90 cm

Bounding Box Volume (x × y × z):
  Volume: (377.66 cm³)

Internal BV/5000 (bv/5000):
  BV/5000: (0.0755)

Actual Part Volume:
  Volume: 103.57 cm³
  Weight: 0.2796 kg

Max Value:
  max(internal_box_v, actual_weight_kg) = 0.2796
  (Comparing 0.0755 vs 0.2796)




In [ ]:

# Example usage
if __name__ == "__main__":
    # Example 1: Calculate both actual and bounding box volumes from STEP file
    step_file = "part.step"  # Replace with your file path
    
    result = process_step_file(step_file, material='aluminum')
    
    if result['success']:
        print(f"File: {result['file']}")
        print(f"Material: {result['material']}")
        print(f"Density: {result['density_g_cm3']} g/cm³")
        print("\n" + "-"*50)
        
        print("\nBounding Box Dimensions:")
        print(f"  X × Y × Z: {result['x_dimension']:.2f} × {result['y_dimension']:.2f} × {result['z_dimension']:.2f} mm")
        
        print("\nBounding Box Volume (x × y × z):")
        print(f"  Volume: {result['bounding_box_volume_mm3']:.2f} mm³ ({result['bounding_box_volume_cm3']:.2f} cm³)")
        print(f"  Weight: {result['bbox_weight_g']:.2f} g ({result['bbox_weight_kg']:.4f} kg)")
        
        print("\nActual Part Volume:")
        print(f"  Volume: {result['actual_volume_mm3']:.2f} mm³ ({result['actual_volume_cm3']:.2f} cm³)")
        print(f"  Weight: {result['actual_weight_g']:.2f} g ({result['actual_weight_kg']:.4f} kg)")
        
        print(f"\nInternal Box Volume (Bounding Box / 5000):")
        print(f"  Internal box_v: {result['internal_box_v']:.2f} mm³")
        
    else:
        print(f"Error: {result['error']}")
    
    print("\n" + "="*50 + "\n")
    
    # Example 2: Calculate volume from manual dimensions (x * y * z)
    length = 100  # mm
    width = 50    # mm
    height = 20   # mm
    
    volume = calculate_volume_xyz(length, width, height)
    weight = calculate_weight(volume, MATERIAL_DENSITIES['steel'])
    internal_box = volume / 5000
    
    print(f"Manual calculation for: {length} × {width} × {height} mm")
    print(f"Volume: {volume:.2f} mm³ ({volume/1000:.2f} cm³)")
    print(f"Internal box_v: {internal_box:.2f} mm³")
    print(f"Material: Steel (7.85 g/cm³)")
    print(f"Weight: {weight['grams']:.2f} g ({weight['kilograms']:.4f} kg)")
    
    print("\n" + "="*50 + "\n")
    
    # Example 3: Batch processing multiple files
    # files = ['part1.step', 'part2.step', 'part3.step']
    # print(f"{'File':<20} {'Actual Vol (cm³)':<18} {'BBox Vol (cm³)':<18} {'Internal box_v':<15} {'Actual Weight (kg)':<20}")
    # print("-" * 95)
    # for file in files:
    #     result = process_step_file(file, material='steel')
    #     if result['success']:
    #         print(f"{file:<20} {result['actual_volume_cm3']:<18.2f} {result['bounding_box_volume_cm3']:<18.2f} {result['internal_box_v']:<15.2f} {result['actual_weight_kg']:<20.4f}")

In [31]:
"""
STEP File Volume Calculator for Metal Parts
Calculates volume and weight from STEP files
"""

from OCP.STEPControl import STEPControl_Reader
from OCP.IFSelect import IFSelect_ReturnStatus
from OCP.GProp import GProp_GProps
from OCP.BRepGProp import BRepGProp
from OCP.TopoDS import TopoDS_Shape
from OCP.Bnd import Bnd_Box
from OCP.BRepBndLib import BRepBndLib
import os
import pandas as pd


def calculate_volume_from_step(step_file_path):
    """
    Calculate the volume of a 3D model from a STEP file.
    
    Args:
        step_file_path (str): Path to the STEP file
        
    Returns:
        float: Volume in cubic millimeters (mm³)
        
    Raises:
        FileNotFoundError: If the STEP file doesn't exist
        ValueError: If the STEP file cannot be read or processed
    """
    step_file_path = str(step_file_path)
    
    if not os.path.exists(step_file_path):
        raise FileNotFoundError(f"STEP file not found: {step_file_path}")
    
    reader = STEPControl_Reader()
    status = reader.ReadFile(step_file_path)
    
    if status != IFSelect_ReturnStatus.IFSelect_RetDone:
        raise ValueError(f"Error reading STEP file: {step_file_path}")
    
    reader.TransferRoots()
    shape = reader.OneShape()
    
    if shape.IsNull():
        raise ValueError("No valid shape found in STEP file")
    
    props = GProp_GProps()
    BRepGProp.VolumeProperties_s(shape, props)
    
    return props.Mass()


def calculate_bounding_box_volume_from_step(step_file_path):
    """
    Calculate the bounding box volume (x × y × z) from a STEP file.
    
    Args:
        step_file_path (str): Path to the STEP file
        
    Returns:
        dict: Dictionary containing bounding box dimensions and volume
    """
    step_file_path = str(step_file_path)
    
    if not os.path.exists(step_file_path):
        raise FileNotFoundError(f"STEP file not found: {step_file_path}")
    
    reader = STEPControl_Reader()
    status = reader.ReadFile(step_file_path)
    
    if status != IFSelect_ReturnStatus.IFSelect_RetDone:
        raise ValueError(f"Error reading STEP file: {step_file_path}")
    
    reader.TransferRoots()
    shape = reader.OneShape()
    
    if shape.IsNull():
        raise ValueError("No valid shape found in STEP file")
    
    bbox = Bnd_Box()
    BRepBndLib.Add_s(shape, bbox)
    
    xmin, ymin, zmin, xmax, ymax, zmax = bbox.Get()
    
    x_dimension = xmax - xmin
    y_dimension = ymax - ymin
    z_dimension = zmax - zmin
    
    bbox_volume = x_dimension * y_dimension * z_dimension
    
    return {
        'x_dimension': x_dimension,
        'y_dimension': y_dimension,
        'z_dimension': z_dimension,
        'bounding_box_volume': bbox_volume
    }


# Common material densities (g/cm³)
MATERIAL_DENSITIES = {
    'aluminum': 2.70,
    'steel': 7.85,
    'stainless_steel': 8.00,
    'brass': 8.50,
    'copper': 8.96,
    'titanium': 4.51,
    'cast_iron': 7.20,
    'bronze': 8.80,
    'zinc': 7.14,
    'magnesium': 1.74,
}


def process_step_file(step_file_path, material='steel'):
    """
    Complete workflow: Read STEP file and calculate volumes.
    Returns a pandas DataFrame with columns: x, y, z, BV/5000, SV, density, max_internal_or_weight
    
    Args:
        step_file_path (str): Path to the STEP file
        material (str): Material type (default: 'steel')
        
    Returns:
        pd.DataFrame: Table with volume and weight information
    """
    try:
        step_file_path = str(step_file_path)
        
        # Calculate actual volume (SV - Solid Volume)
        actual_volume_mm3 = calculate_volume_from_step(step_file_path)
        sv_cm3 = actual_volume_mm3 / 1000.0  # Convert to cm³
        
        # Calculate bounding box volume
        bbox_info = calculate_bounding_box_volume_from_step(step_file_path)
        bbox_volume_mm3 = bbox_info['bounding_box_volume']
        
        # Get dimensions
        x = bbox_info['x_dimension']
        y = bbox_info['y_dimension']
        z = bbox_info['z_dimension']
        
        # Calculate BV/5000 (internal box volume)
        bv_5000 = bbox_volume_mm3 / 5000000  # Convert to cm³ and divide by 5000
        
        # Get material density
        if isinstance(material, str):
            density = MATERIAL_DENSITIES.get(material.lower(), MATERIAL_DENSITIES['steel'])
        else:
            density = material
        
        # Calculate actual weight in kg (SV * density)
        actual_weight_kg = sv_cm3 * density / 1000.0
        
        # Calculate max(internal_box_v, actual_weight_kg)
        max_value = max(bv_5000, actual_weight_kg)
        
        # Create DataFrame
        df = pd.DataFrame({
            'x': [x],
            'y': [y],
            'z': [z],
            'BV/5000': [bv_5000],
            'SV': [sv_cm3],
            'density': [density],
            'SV_weight_kg': [actual_weight_kg],
            'max_internal_or_weight': [max_value]
        })
        
        return df
    
    except Exception as e:
        print(f"Error processing {step_file_path}: {str(e)}")
        return None


def process_multiple_step_files(step_file_paths, material='steel'):
    """
    Process multiple STEP files and combine results into a single table.
    
    Args:
        step_file_paths (list): List of paths to STEP files
        material (str): Material type (default: 'steel')
        
    Returns:
        pd.DataFrame: Combined table with all results
    """
    results = []
    
    for file_path in step_file_paths:
        df = process_step_file(file_path, material)
        if df is not None:
            results.append(df)
    
    if results:
        return pd.concat(results, ignore_index=True)
    else:
        return pd.DataFrame(columns=['x', 'y', 'z', 'BV/5000', 'SV', 'density','SV_weight_kg', 'max_internal_or_weight'])



In [33]:

result_table = process_step_file("101833-001-A.step", material='aluminum')
result_table


,x,y,z,BV/5000,SV,density,SV_weight_kg,max_internal_or_weight
0,118.618901,111.633901,3.200501,0.008476,33.627534,2.7,0.090794,0.090794
